# Data Preprocessing

In [1]:
# import pandas as pd 
# from scipy import stats
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
# from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

# Statistical libraries
from scipy import stats
from scipy.stats import zscore, skew

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


Libraries imported successfully!


In [18]:
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder

## Data Cleaning

In [4]:
train_url = r'https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_train.csv'
test_url = r'https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_test.csv'

In [5]:
test_data = pd.read_csv(test_url, sep=",")
test_data.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban


In [6]:
train_data = pd.read_csv(train_url, sep=",")
train_data.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [7]:
df = train_data.copy()

In [8]:
#Correcting datatypes
# df["Dependents"] = df["Dependents"].replace("3+", 3)

df= df.astype({"ApplicantIncome":"float64"})

In [9]:
#Checking missing values
print("Missing Values")
missing_values = df.isnull().sum()
if missing_values.sum()>0:
    print(missing_values[missing_values > 0])
else:
    print("No missing values")

#Checking for duplicates
print("\nDuplicate Rows:")
duplicates = df.duplicated().sum()
print(f"Number of duplicated Rows: {duplicates}")

if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates/len(duplicates))*100:.2f}%")
else:
    print("There are no duplicate rows.")

Missing Values
Gender              13
Married              3
Dependents          15
Self_Employed       32
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
dtype: int64

Duplicate Rows:
Number of duplicated Rows: 0
There are no duplicate rows.


In [10]:
#Handling missing values
df.dropna(subset=["LoanAmount"], inplace=True)

In [11]:
df.dropna(subset=["Loan_Amount_Term"], inplace=True)

In [12]:
#Handling missing values
df["Gender"] = df['Gender'].fillna(df["Gender"].mode().iloc[0])
df["Married"] = df['Married'].fillna(df["Married"].mode().iloc[0])
df["Self_Employed"] = df['Self_Employed'].fillna(df["Self_Employed"].mode().iloc[0])
df["Dependents"] = df['Dependents'].fillna(df["Dependents"].mode().iloc[0])

df["Credit_History"] = df['Credit_History'].fillna(df["Credit_History"].median())


In [19]:
be = BinaryEncoder()
le = LabelEncoder()

In [21]:
df["Loan_Status"] = le.fit_transform(df["Loan_Status"])
df[["Loan_Status"]]

,Loan_Status
1,0
2,1
3,1
4,1
5,1
...,...
609,1
610,1
611,1
612,1


In [ ]:
# df["Loan_Status_Int"] = (df["Loan_Status"] == 'Y').astype(int)

In [ ]:
def map_credit_history_category(c):
    """
    Converts credit history to a categorical feature
    """
    if c == 1:
        return "Good"
    else:
        return "Bad"

In [ ]:
df["Credit_History_Label"] = df["Credit_History"].apply(map_credit_history_category)

In [ ]:
df.isnull().sum()

Loan_ID                 0
Gender                  0
Married                 0
Dependents              0
Education               0
Self_Employed           0
ApplicantIncome         0
CoapplicantIncome       0
LoanAmount              0
Loan_Amount_Term        0
Credit_History          0
Property_Area           0
Loan_Status             0
Loan_Status_Int         0
Credit_History_Label    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578 entries, 1 to 613
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Loan_ID               578 non-null    object 
 1   Gender                578 non-null    object 
 2   Married               578 non-null    object 
 3   Dependents            578 non-null    object 
 4   Education             578 non-null    object 
 5   Self_Employed         578 non-null    object 
 6   ApplicantIncome       578 non-null    float64
 7   CoapplicantIncome     578 non-null    float64
 8   LoanAmount            578 non-null    float64
 9   Loan_Amount_Term      578 non-null    float64
 10  Credit_History        578 non-null    float64
 11  Property_Area         578 non-null    object 
 12  Loan_Status           578 non-null    object 
 13  Loan_Status_Int       578 non-null    int64  
 14  Credit_History_Label  578 non-null    object 
dtypes: float64(5), int64(1), obj

## 2. EDA-Based Data Quality Assessment

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578 entries, 1 to 613
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Loan_ID               578 non-null    object 
 1   Gender                578 non-null    object 
 2   Married               578 non-null    object 
 3   Dependents            578 non-null    object 
 4   Education             578 non-null    object 
 5   Self_Employed         578 non-null    object 
 6   ApplicantIncome       578 non-null    float64
 7   CoapplicantIncome     578 non-null    float64
 8   LoanAmount            578 non-null    float64
 9   Loan_Amount_Term      578 non-null    float64
 10  Credit_History        578 non-null    float64
 11  Property_Area         578 non-null    object 
 12  Loan_Status           578 non-null    object 
 13  Loan_Status_Int       578 non-null    int64  
 14  Credit_History_Label  578 non-null    object 
dtypes: float64(5), int64(1), obj

In [ ]:
df_processed = df.copy()

# 1. Checking for missing values
print("\n1. Missing Values:")
missing_values = df_processed.isnull().sum()
if missing_values.sum()>0:
    print(missing_values[missing_values>0])
else:
    print("No missing values found (as expected from EDA)")

#2. Checking for duplicates
print("\n2. Duplicate Rows:")
duplicates = df_processed.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates/len(df_processed))*100:.2f}%")

#3. Checking for skewness for variable identified in EDA as right-skewed
print("\n3. Skewness Analysis (EDA identified right-skewed variables):")
skewed_vars = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "LoanAmountTerm"]
for var in skewed_vars:
    if var in df_processed.columns:
        skewness = skew(df_processed[var])
        print(f"{var}: skewness = {skewness:.3f} ({'right-skewed' if skewness >0.5 else 'approximately normal'})")

#4. Checking for correlation with target (EDA evidence)
print("\n4. Correlation with Loan_Status (EDA Evidence):")
num_features = df_processed.select_dtypes(include=['float64', 'int64']).columns
num_features = num_features.drop("Credit_History")
num_features = pd.DataFrame(df_processed[num_features])

correlations = num_features.corr()["Loan_Status_Int"].sort_values(key=abs, ascending=False)
high_signal = correlations[abs(correlations)>0.2].drop("Loan_Status_Int")
for feature, corr in high_signal.items():
    print(f"{feature}: {corr:.3f}")

print("\nLow-signal features (|correlation| < 0.1):")
low_signal = correlations[abs(correlations) < 0.1]
for feature, corr in low_signal.items():
    print(f"{feature}: {corr:.3f}")


1. Missing Values:
No missing values found (as expected from EDA)

2. Duplicate Rows:
Number of duplicate rows: 0

3. Skewness Analysis (EDA identified right-skewed variables):
ApplicantIncome: skewness = 6.494 (right-skewed)
CoapplicantIncome: skewness = 7.399 (right-skewed)
LoanAmount: skewness = 2.643 (right-skewed)

4. Correlation with Loan_Status (EDA Evidence):

Low-signal features (|correlation| < 0.1):
CoapplicantIncome: -0.070
LoanAmount: -0.042
Loan_Amount_Term: -0.030
ApplicantIncome: -0.006


In [ ]:
# num_features = df.select_dtypes(include=['float64', 'int64']).columns
# num_features = num_features.drop("Credit_History")
# num_features = pd.DataFrame(df[num_features])
# num_features

In [ ]:
df_processed.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'Loan_Status_Int', 'Credit_History_Label'],
      dtype='object')

## 3. Handle Duplicates

In [ ]:
if duplicates > 0:
    print(f"Removing {duplicates} duplicate rows...")
    df_processed = df_processed.drop_duplicates()
    print(f"Dataset shape after removing duplicates: {df_processed.shape}")
else:
    print("No duplicates to remove (as expected from EDA)")

No duplicates to remove (as expected from EDA)


## 4. Log-Transform Skewed Variables (EDA Recommended)

In [ ]:
print("=== LOG-TRANSFORMING SKEWED VARIABLES ===")
print("EDA identified these variables as right-skewed and recommended log transformation:")

#Variables to log-transform based on EDA findings
skewed_vars = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "LoanAmountTerm"]

for var in skewed_vars:
    if var in df_processed.columns:
        #Check if variable has zero or negative values
        min_val = df_processed[var].min()
        if min_val <= 0:
            #Use log1p for variables with zeros
            df_processed[f"{var}_log"] = np.log1p(df_processed[var])
            print(f"\n{var}: Applied log1p transformation (had {min_val:.3f} minimum value)")
        else:
            #use log for positive values only
            df_processed[f"{var}_log"]= np.log(df_processed[var])
            print(f"\n{var}: Applied log transformation")
        
        #Checking skewness before and after
        original_skew = skew(df_processed[var])
        transformed_skew = skew(df_processed[f"{var}_log"])
        print(f"Original skewness: {original_skew:.3f} → Transformed skewness: {transformed_skew:.3f}")

print(f"\n Dataset shape after log transformation: {df_processed.shape}")
print(f"New log-transformed columns:", [col for col in df_processed.columns if '_log' in col])

=== LOG-TRANSFORMING SKEWED VARIABLES ===
EDA identified these variables as right-skewed and recommended log transformation:

ApplicantIncome: Applied log transformation
Original skewness: 6.494 → Transformed skewness: 0.465

CoapplicantIncome: Applied log1p transformation (had 0.000 minimum value)
Original skewness: 7.399 → Transformed skewness: -0.178

LoanAmount: Applied log transformation
Original skewness: 2.643 → Transformed skewness: -0.207

 Dataset shape after log transformation: (578, 18)
New log-transformed columns: ['ApplicantIncome_log', 'CoapplicantIncome_log', 'LoanAmount_log']


## 5. Outlier Treatment (EDA Recommendation)


Bsed on EDA findings, handle outliers using IQR-capping method

In [ ]:
# Outlier treatment based on EDA recommendations
print("=== OUTLIER TREATMENT (IQR-CAPPING METHOD) ===")
print("EDA recommended IQR-capping for extreme features to preserve data points")

numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
if "Loan_Status_Int" in numerical_cols:
    numerical_cols.remove("Loan_Status_Int")
    numerical_cols.remove("Credit_History")
print(f"Treating outliers in {len(numerical_cols)} numerical features...")

#Applying IQR-capping method
outliers_capped = 0
for col in numerical_cols:
    Q1 = df_processed[col].quantile(0.25)
    Q3 = df_processed[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5 * IQR

    #Counting outliers before capping
    outliers_before = ((df_processed[col]< lower_bound)| (df_processed[col] > upper_bound)).sum()
    if outliers_before > 0:
        #Cap outliers
        df_processed[col] = np.where(df_processed[col] < lower_bound, lower_bound, df_processed[col])
        df_processed[col] = np.where(df_processed[col]> upper_bound, upper_bound, df_processed[col])
        outliers_capped += outliers_before
        print(f"\n{col}: Capped {outliers_before} outliers")

print(f'\nTotal outliers capped: {outliers_capped}')
print(f"Dataset shape after outlier treatment: {df_processed.shape}")

=== OUTLIER TREATMENT (IQR-CAPPING METHOD) ===
EDA recommended IQR-capping for extreme features to preserve data points
Treating outliers in 7 numerical features...

ApplicantIncome: Capped 48 outliers

CoapplicantIncome: Capped 18 outliers

LoanAmount: Capped 39 outliers

Loan_Amount_Term: Capped 85 outliers

ApplicantIncome_log: Capped 27 outliers

LoanAmount_log: Capped 34 outliers

Total outliers capped: 251
Dataset shape after outlier treatment: (578, 18)


## 6. Feature Engineering

In [ ]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'Loan_Status_Int', 'Credit_History_Label'],
      dtype='object')

In [ ]:
print("FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS")

#Total loan applicant income
df["Total_Income"] = df["ApplicantIncome"] + df["CoapplicantIncome"]

#Loan to income ratio
df["Loan_Income_Ratio"] = df["LoanAmount"]/df["Total_Income"]

#Estimated monthly payment plan
df["Monthly_Payment"] = df["LoanAmount"]/df["Loan_Amount_Term"]

#Splitting income into bins

df[["Total_Income", "LoanAmount","Loan_Income_Ratio", "Monthly_Payment"]].head()

FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS


,Total_Income,LoanAmount,Loan_Income_Ratio,Monthly_Payment
1,6091.0,128.0,0.021015,0.355556
2,3000.0,66.0,0.022000,0.183333
3,4941.0,120.0,0.024287,0.333333
4,6000.0,141.0,0.023500,0.391667
5,9613.0,267.0,0.027775,0.741667


## 7. Feature Selection


In [ ]:
df_processed

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Loan_Status_Int,Credit_History_Label,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
1,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,Rural,N,0,Good,8.430109,7.319202,4.852030
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y,1,Good,8.006368,0.000000,4.189655
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y,1,Good,7.856707,7.765993,4.787492
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y,1,Good,8.699515,0.000000,4.948760
5,LP001011,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.0,360.0,1.0,Urban,Y,1,Good,8.597297,8.342125,5.587249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900.0,0.0,71.0,360.0,1.0,Rural,Y,1,Good,7.972466,0.000000,4.262680
610,LP002979,Male,Yes,3+,Graduate,No,4106.0,0.0,40.0,360.0,1.0,Rural,Y,1,Good,8.320205,0.000000,3.826979
611,LP002983,Male,Yes,1,Graduate,No,8072.0,240.0,253.0,360.0,1.0,Urban,Y,1,Good,8.996157,5.484797,5.533389
612,LP002984,Male,Yes,2,Graduate,No,7583.0,0.0,187.0,360.0,1.0,Urban,Y,1,Good,8.933664,0.000000,5.231109


In [ ]:
# Feature selection based on EDA correlation evidence
print("=== FEATURE SELECTION BASED ON EDA CORRELATION EVIDENCE ===")

X = df_processed.drop("Loan_Status" axis=1)

=== FEATURE SELECTION BASED ON EDA CORRELATION EVIDENCE ===


TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

## Preprocessing

Encode Gender

In [ ]:
#This a binary category (we can have label or binary encoding)

le = LabelEncoder()
df["gender_encoded"] = le.fit_transform(df["Gender"])

#Viewing the encoded data against the actual data
df[["gender_encoded", "Gender"]]

NameError: name 'LabelEncoder' is not defined

In [ ]:
df['Education'].unique()

array(['Graduate', 'Not Graduate'], dtype=object)